In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt
import ESN_Modules as ESN
from scipy import linalg
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

d = ESN.reshape(numpy.loadtxt("NormMackey22.txt", delimiter = ","))
d = d[0:10000, :]
numpy.savetxt("series_mackey22_norm.txt", d, delimiter = ",")

In [2]:
def train_parameters(X, train_length, transiente, N, K, scale, radius, sparsity, leak):
    numpy.random.seed(X)
    # PESOS DE ENTRADA
    W_input = ESN.input_weights(N = N, K = K, input_scaling = scale)
    # PESOS DE RESERVATORIO
    W_reservoir = ESN.reservoir_weights(N = N, spectral_radius = radius, sparsity = sparsity)

    # MATRIZ DE ESTADOS (TRAINNING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = train_length - transiente), 
                                     input_patterns = train_dataset, 
                                     win = W_input, 
                                     wres = W_reservoir, 
                                     initial_state = numpy.zeros((N, 1)),
                                     leaking = leak, 
                                     transiente = transiente)

    # PESOS DE SAÍDA
    W_output = ESN.ridge_regression(states = state_matrix, target = train_target)
    # ERRO DE TREINO
    return(numpy.dot(W_output, state_matrix))

In [3]:
# TAMANHO DE JANELA DE ENTRADA (INPUT)
len_window_input = 1
# TAMANHO DE JANELA DE SAÍDA (OUTPUT)
len_window_output = 1

# PASSOS AHEAD QUE DESEJAMOS PREVER
steps = 1

# TAMANHO DA JANELA DE TRAINNING
train_length = 3500
# TAMANHO DA JANELA DE TESTING
test_length = 1500

# TRANSIENTE, VALORES A SEREM DESCARTADOS
transiente = int(train_length*0.1)

# NÚMERO DE NEURÔNIOS NA CAMADA OCULTA, INT, N > 0
neurons = [20]

# NÚMERO DE HIPERPARÂMETROS A SEREM TESTADOS POR CATEGORIA
spam = 10
# NÚMERO DE RODADAS
rounds = 10

# TAXA DE VAZAMENTO ESPECTRAL, FLOAT, NO CONJUNTO (0, 1]
leaking_rates = numpy.linspace(start = 0.05, stop = 0.95, num = spam).round(2)
# ESPARSIDADE DA MATRIZ, FLOAT, NO CONJUNTO [0, 1]
sparsities = numpy.linspace(start = 0.05, stop = 0.95, num = spam).round(2)
# ESCALA DOS DADOS DE ENTRADA, FLOAT, NO CONJUNTO (0, INF)
input_scalings = numpy.linspace(start = 0.1, stop = 1.9, num = spam).round(2)
# RAIO ESPECTRAL DESEJADO, FLOAT, NO CONJUNTO (0, INF)
spectral_radius = numpy.linspace(start = 0.1, stop = 1.9, num = spam).round(2)

In [4]:
# DEFINE UMA SEMENTE DO GERADOR PSEUDO-ALEATORIO
numpy.random.seed(42)

cols = ["ROUND", "WINDOW", "NEURONS", "LEAK", "SPARSITY", "SCALE", "RADIUS", "MSE", "RMSE", "MAPE", 
        "MSE(X)", "RMSE(X)", "MAPE(X)", "MSE(Y)", "RMSE(Y)", "MAPE(Y)", "MSE(Z)", "RMSE(Z)", "MAPE(Z)",
        "FRAMEWORK"]

sample = (spam**4) * len(neurons) * rounds
architectures = numpy.random.randint(low = 1, high = sample * 2, size = sample)
simulation = pandas.DataFrame(numpy.nan, columns = cols, 
                              index = numpy.linspace(start = 0, stop = sample - 1, num = sample).astype(int))
i = 0

noise_weight = 0.0025

# OUTPUT ESPERADO
output_dataset, L = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_output, begin = len_window_input)
#output_dataset += numpy.random.normal(0, noise_weight, output_dataset.shape)
# DATASET DE TREINO (TARGET) -> [(Ysteps+transiente), (Ysteps+transiente+1), ..., (Ysteps+janela_treino-1)]
train_target = output_dataset[None, steps + transiente - 1 : train_length + steps - 1, :]
# DATASET DE TESTE (TARGET) -> [(Ysteps+janela_treino+janela_teste), ..., (Ysteps+janela_treino+janela_teste-1)]
test_target = output_dataset[train_length + steps - 1 : train_length + test_length + steps - 1, :].T

eval_model = train_target[0].T

for rnd in range(rounds):
    print(rnd)
    for N in neurons:
        for leak in leaking_rates:
            for sparsity in sparsities:
                for scale in input_scalings:
                    for radius in spectral_radius:
                        # INPUT DO SISTEMA
                        numpy.random.seed(architectures[i])
                        input_dataset, K = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_input, begin = 0)
                        input_dataset += numpy.random.normal(0, noise_weight, input_dataset.shape)
                        
                        # DATASET DE TREINO -> [(X0), (X1), (X2), ..., (Xjanela_treino-1)]
                        train_dataset = input_dataset[0 : train_length, :]
                        

                        # DATASET DE TESTE -> [(Xjanela_treino), (Xjanela_treino+1), (Xjanela_treino+2), ..., (Xjanela_treino+janela_teste-1)]
                        test_dataset = input_dataset[train_length : train_length + test_length, :]
                        
                        
                        predict = train_parameters(
                            X = architectures[i], 
                            train_length = train_length, 
                            transiente = transiente, 
                            N = N, 
                            K = K, 
                            scale = scale, 
                            radius = radius, 
                            sparsity = sparsity, 
                            leak = leak
                        )
                        simulation.iloc[i,:] = [rnd, len_window_input, N, leak, sparsity, scale, radius] + ESN.model_metrics(eval_model, predict) + [architectures[i]]
                        i += 1

0
1
2
3
4
5
6
7
8
9


In [5]:
simulation

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
0,0.0,1.0,20.0,0.05,0.05,0.1,0.1,0.008237,0.073687,0.543171,0.008237,0.073687,0.543171,NaN,NaN,NaN,NaN,NaN,NaN,121959.0
1,0.0,1.0,20.0,0.05,0.05,0.1,0.3,0.007310,0.067999,0.478986,0.007310,0.067999,0.478986,NaN,NaN,NaN,NaN,NaN,NaN,146868.0
2,0.0,1.0,20.0,0.05,0.05,0.1,0.5,0.006298,0.062065,0.413463,0.006298,0.062065,0.413463,NaN,NaN,NaN,NaN,NaN,NaN,131933.0
3,0.0,1.0,20.0,0.05,0.05,0.1,0.7,0.005219,0.055598,0.312157,0.005219,0.055598,0.312157,NaN,NaN,NaN,NaN,NaN,NaN,103695.0
4,0.0,1.0,20.0,0.05,0.05,0.1,0.9,0.005342,0.057158,0.313407,0.005342,0.057158,0.313407,NaN,NaN,NaN,NaN,NaN,NaN,119880.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,9.0,1.0,20.0,0.95,0.95,1.9,1.1,0.000353,0.014096,0.072441,0.000353,0.014096,0.072441,NaN,NaN,NaN,NaN,NaN,NaN,132650.0
99996,9.0,1.0,20.0,0.95,0.95,1.9,1.3,0.000428,0.015900,0.071345,0.000428,0.015900,0.071345,NaN,NaN,NaN,NaN,NaN,NaN,37066.0
99997,9.0,1.0,20.0,0.95,0.95,1.9,1.5,0.000399,0.015006,0.071820,0.000399,0.015006,0.071820,NaN,NaN,NaN,NaN,NaN,NaN,39484.0
99998,9.0,1.0,20.0,0.95,0.95,1.9,1.7,0.000467,0.015637,0.077241,0.000467,0.015637,0.077241,NaN,NaN,NaN,NaN,NaN,NaN,36202.0


In [6]:
simulation[(simulation.LEAK == 0.05) & (simulation.SCALE == 0.1) & (simulation.SPARSITY == 0.55) & (simulation.RADIUS == 0.5)]

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
502,0.0,1.0,20.0,0.05,0.55,0.1,0.5,0.007053,0.066440,0.459954,0.007053,0.066440,0.459954,NaN,NaN,NaN,NaN,NaN,NaN,113430.0
10502,1.0,1.0,20.0,0.05,0.55,0.1,0.5,0.007007,0.066031,0.459187,0.007007,0.066031,0.459187,NaN,NaN,NaN,NaN,NaN,NaN,156098.0
20502,2.0,1.0,20.0,0.05,0.55,0.1,0.5,0.007306,0.067717,0.479578,0.007306,0.067717,0.479578,NaN,NaN,NaN,NaN,NaN,NaN,139292.0
30502,3.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006524,0.063038,0.436878,0.006524,0.063038,0.436878,NaN,NaN,NaN,NaN,NaN,NaN,39769.0
40502,4.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006787,0.064457,0.443759,0.006787,0.064457,0.443759,NaN,NaN,NaN,NaN,NaN,NaN,80616.0
50502,5.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006502,0.063063,0.425100,0.006502,0.063063,0.425100,NaN,NaN,NaN,NaN,NaN,NaN,23793.0
60502,6.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006048,0.059455,0.416341,0.006048,0.059455,0.416341,NaN,NaN,NaN,NaN,NaN,NaN,152955.0
70502,7.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006463,0.062697,0.428602,0.006463,0.062697,0.428602,NaN,NaN,NaN,NaN,NaN,NaN,83533.0
80502,8.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006871,0.065508,0.442202,0.006871,0.065508,0.442202,NaN,NaN,NaN,NaN,NaN,NaN,5988.0
90502,9.0,1.0,20.0,0.05,0.55,0.1,0.5,0.006752,0.064458,0.444869,0.006752,0.064458,0.444869,NaN,NaN,NaN,NaN,NaN,NaN,6070.0


In [7]:
architectures

array([121959, 146868, 131933, ...,  39484,  36202, 163726])

In [8]:
simulation.to_csv("ParametrosTreino_NoiseMackey22.txt", index = False)